In [19]:
import transformers
from transformers import AutoTokenizer, AutoModel

In [20]:
qar_tokenizer = AutoTokenizer.from_pretrained('yjernite/retribert-base-uncased')
qar_model = AutoModel.from_pretrained('yjernite/retribert-base-uncased').to('cuda')
_ = qar_model.eval()

In [21]:
!pip install langchain

In [22]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

from IPython.display import display, HTML
import json
import time
import pathlib

In [23]:
!pip install sentence-transformers
!pip install faiss-gpu

In [24]:
loader = DirectoryLoader("/kaggle/input/context", glob="*.txt", loader_cls=TextLoader)

# interpret information in the documents
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
texts = splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})

# create and save the local database
db = FAISS.from_documents(texts, embeddings)
db.save_local("faiss")

In [25]:
question="How does mahe rank?"
ans=db.max_marginal_relevance_search(question,k=2, fetch_k=3)

In [26]:
excerpts=[]
for i in ans:
    excerpts.append(i.page_content)
context=''
for i in excerpts:
    context=context+i
context

'MAHE has a dedicated placement cell that provides guidance and assistance to the students for their career development and placement. MAHE has a strong industry-academia interface, with regular interactions, workshops, seminars, and internships with leading companies and organizations. MAHE attracts over 250 recruiters every year, including 50 ‘Fortune 500’ companies, and has a high percentage of core and product-based placements. MAHE also has a global alumni network, with over 1.5 lakh alumniMAHE is an Institution of Eminence Deemed to be University established in the year 19531. It is one of India’s leading academic and research institutions, with a rank of 6th among top universities in India according to NIRF Ranking 20231. It has a campus in Manipal, Karnataka, and a constituent college in Sikkim2. It offers 300+ programs in 30+ disciplines, including engineering, medical, MBA, pharmacy, fashion design and architecture1. It has collaborations with leading research laboratories'

In [27]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
qamodel = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')

In [28]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

answer=question_answerer(question, context)

print(answer['answer'])

6th


In [31]:
def find_sentence(context, s, e):
    start=s
    while(start>=0):
        if context[start]=='.':
            break
        start=start-1
    end=e
    while(end<len(context)):
        if context[end]=='.':
            break
        end=end+1
    return start, end

In [34]:
 def answer_question(question):
    ans=db.max_marginal_relevance_search(question,k=2, fetch_k=3)
    excerpts=[]
    for i in ans:
        excerpts.append(i.page_content)
    context=''
    for i in excerpts:
        context=context+i
    answer=question_answerer(question, context)
    start, end=find_sentence(context, answer['start'], answer['end'])
    print(context[start+1:end+1])
    #print(answer['answer'])

In [33]:
question=''
while question!='-1':
    question=input('What would you like to know?:')
    answer_question(question)

What would you like to know?: how does mahe rank


. It is one of India’s leading academic and research institutions, with a rank of 6th among top universities in India according to NIRF Ranking 20231
6th


What would you like to know?: where is mahe


. MAHE also provides a safe and secure campus, with 24/7 security, CCTV surveillance, and emergency services1
safe and secure campus


What would you like to know?: where are mahe campuses


. It has a campus in Manipal, Karnataka, and a constituent college in Sikkim2
Manipal, Karnataka


What would you like to know?: what courses does mahe offer


. It offers 300+ programs in 30+ disciplines, including engineering, medical, MBA, pharmacy, fashion design and architecture1
engineering, medical, MBA, pharmacy, fashion design and architecture1


What would you like to know?: -1


. Teams from ManipalTech have won innovation and entrepreneurship competitions such as the GE Edison Innovation ChallengeMood Indigo, IIT Bombay
GE Edison Innovation ChallengeMood Indigo
